In [1]:
import os
import sys
import math
import logging
import structlog
from pathlib import Path
import json
import re
from itertools import product, combinations

import tomli
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={"figure.figsize": (12, 6.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

In [2]:
logging.basicConfig(level=logging.WARNING, stream=sys.stdout)

In [4]:
import pytanis
from pytanis import GSheetClient, PretalxClient
from pytanis.google import Scope, gsheet_rows_for_fmt, worksheet_range
from pytanis.review import read_assignment_as_df, save_assignments_as_json, Col
from pytanis.pretalx import subs_as_df, reviews_as_df, speakers_as_df
from pytanis.utils import implode


In [5]:
# Be aware that this notebook might only run with the following version
pytanis.__version__ 

'0.4.post1.dev5+g2cf4040.d20230325'

In [51]:
# Import event-specific settings to don't have them here in the notebook
with open('config.toml', 'rb') as fh:
    cfg = tomli.load(fh)


## Gather all Data, i.e. Submissions, Votes and Scheduling Data

In [33]:
pretalx_client = PretalxClient(blocking=True)
talks_count, talks = pretalx_client.submissions(cfg['event_name'], params={"questions": "all", "state": ["confirmed"]})
talks = list(talks)
rooms_count, rooms = pretalx_client.rooms(cfg['event_name'])
rooms = list(rooms)

  0%|          | 0/116 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [44]:
str(talks[0].slot.start.time())

'15:45:00'

In [48]:
def to_pdf(subs):
    rows = []
    for sub in subs:
        row = {
            Col.submission: sub.code,
            Col.speaker_name: ", ".join([speaker.name for speaker in sub.speakers]),
            "Day": f"{sub.slot.start.day}",
            "Time": f"{sub.slot.start.time()}-{sub.slot.end.time()}",
            "Room": sub.slot.room.en,
            Col.duration: sub.duration,
            Col.track: sub.track.en if sub.track else None,
            Col.title: sub.title,
            
        }
        rows.append(row)
    return pd.DataFrame(rows)

In [49]:
df = to_pdf(talks)

## Upload to GSheet

In [52]:
gsheet_client = GSheetClient(read_only=False)

In [54]:
gsheet_client.save_df_as_gsheet(df, cfg['schedule_spread_id'], cfg['agenda_work_name'])

In [217]:
from gspread_formatting import set_column_width, set_row_height, cellFormat, format_cell_range

worksheet = gsheet_client.gsheet(cfg['schedule_spread_id'], cfg['schedule_work_name'])
set_column_width(worksheet, 'D:J', 200)
set_row_height(worksheet, f"2:{len(timetable_df)+1}", 130)

fmt = cellFormat(wrapStrategy='WRAP')
format_cell_range(worksheet, worksheet_range(worksheet), fmt)

{'spreadsheetId': '1_ZZ6ybY2M6U5pfR34OyKgCmadWE5sX1eTVp2B_DLqmA',
 'replies': [{}]}

## References
* https://fran-espiga.medium.com/mixed-integer-programming-for-time-table-scheduling-eee326deda75
* https://towardsdatascience.com/schedule-optimisation-using-linear-programming-in-python-9b3e1bc241e1
* https://math.stackexchange.com/questions/432003/converting-absolute-value-program-into-linear-program